In [1]:
from data_preprocess.image_matching_dataset import ImageMatchingDataset
import torchvision.transforms as transforms
import torch

from feature_descriptors.sift_descriptor import get_keypoint_and_descriptor
from feature_matching.flann_matcher import flann_matcher
from load_h5py_files import load_sift_output, load_flann_output, load_flann_from_images_name
from clustering.cluster_images import build_graph, graph_clustering
from batch_descriptor import batch_feature_descriptor
from torch.utils.data import DataLoader

import os

In [2]:
from torchvision.transforms.functional import pad

class PadToSize():
    def __init__(self, size, fill=0):
        self.size = size
        self.fill = fill

    def __call__(self, img):
        w, h = img.size
        pad_h = max(0, self.size - h)
        pad_w = max(0, self.size - w)
        padding = [pad_w//2, pad_h//2, pad_w - pad_w//2, pad_h - pad_h//2]
        return pad(img, padding, fill=self.fill)

        
'''Original dataset, no resizing performed, does not work with dataloader since it requires all images of same dimensions'''
train_dataset = ImageMatchingDataset(labels_path='data/train_labels.csv', root_dir='data/train',
                               transform=transforms.Compose([
                                   transforms.ToTensor()
                               ]))

'''train_dataset_resize = ImageMatchingDataset(labels_path='data/train_labels.csv', root_dir='data/train',
                               transform=transforms.Compose([
                                   transforms.Resize((1024, 1024)),
                                   transforms.Pad((0, 0, 1024, 1024), fill=0),
                                   transforms.ToTensor()
                               ]))'''


'''Dataset with resizing and padding if needed, to be used with dataloader to work in batches'''
train_dataset_eq_size = ImageMatchingDataset(labels_path='data/train_labels.csv', root_dir='data/train',
                               transform=transforms.Compose([
                                   PadToSize(1024, fill=0),
                                   transforms.Resize((1024, 1024), antialias=True),
                                   transforms.ToTensor()
                               ]))

In [24]:
train_dataset_eq_size[0]

{'image': tensor([[[0.6902, 0.6863, 0.6902,  ..., 0.2980, 0.2941, 0.2941],
          [0.6863, 0.6824, 0.6784,  ..., 0.3020, 0.2941, 0.2941],
          [0.7333, 0.7294, 0.7294,  ..., 0.3294, 0.3294, 0.3373],
          ...,
          [0.3647, 0.3725, 0.3686,  ..., 0.5137, 0.4824, 0.5412],
          [0.3451, 0.3569, 0.3608,  ..., 0.5137, 0.5059, 0.4784],
          [0.3255, 0.3451, 0.3608,  ..., 0.5333, 0.5176, 0.4078]],
 
         [[0.7059, 0.7020, 0.7059,  ..., 0.4392, 0.4353, 0.4353],
          [0.6980, 0.6902, 0.6902,  ..., 0.4431, 0.4353, 0.4353],
          [0.7412, 0.7373, 0.7373,  ..., 0.4706, 0.4706, 0.4784],
          ...,
          [0.5020, 0.5098, 0.5059,  ..., 0.5059, 0.4745, 0.5333],
          [0.4824, 0.4941, 0.4980,  ..., 0.5059, 0.4980, 0.4706],
          [0.4627, 0.4824, 0.4980,  ..., 0.5255, 0.5098, 0.4000]],
 
         [[0.7020, 0.6980, 0.7020,  ..., 0.1569, 0.1529, 0.1529],
          [0.6941, 0.6863, 0.6863,  ..., 0.1608, 0.1529, 0.1529],
          [0.7373, 0.7333, 0.73

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

DEVICE = torch.device(0 if torch.cuda.is_available() else 'cpu')
train_loader = DataLoader(train_dataset_eq_size, batch_size=1, shuffle=True, num_workers=10, pin_memory=True)

In [18]:
batch_feature_descriptor(train_loader, DEVICE, "sift", "evaluation/sift_descriptors_outputs/")

In [27]:
import h5py

def print_h5_structure(name, obj):
    print(name)

with h5py.File("evaluation/sift_descriptors_outputs/0_1024_sift.h5", "r") as f:
    f.visititems(print_h5_structure)
    print(f["descriptors"][:])
    print(f["keypoints"][:])
    print(f["image_name"].asstr()[:])
    print(f["dataset_name"].asstr()[:])
    print(f['scene_name'].asstr()[:])

dataset_name
descriptors
image_name
image_path
keypoint_scores
keypoints
scene_name
[[[0.15955846 0.07338621 0.03498551 ... 0.02709965 0.05419931 0.09123103]
  [0.02835608 0.02315265 0.05177089 ... 0.06945793 0.04331463 0.04331463]
  [0.11931757 0.15133567 0.03047096 ... 0.03518483 0.02487943 0.01759242]
  ...
  [0.05027858 0.00099999 0.02687503 ... 0.04654892 0.02687503 0.0600944 ]
  [0.06653827 0.00099999 0.00099999 ... 0.06872043 0.01718011 0.06653827]
  [0.0734479  0.06304731 0.05588546 ... 0.06526016 0.03767797 0.02918523]]]
[[[585.6262  571.4224 ]
  [583.6034  665.56195]
  [583.29895 501.2842 ]
  ...
  [345.56717 623.4795 ]
  [666.94116 831.7335 ]
  [465.9309  668.9876 ]]]
['peach_0058.png']
['amy_gardens']
['peach']
